In [1]:
# Setup API
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("RB_Vehicle_1")


In [2]:
# Setup môi trường
# Cài đặt các gói, NHƯNG ép numpy phải ở phiên bản 1.x
!pip install "rfdetr[metrics]" roboflow "numpy<2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Check gpu
!nvidia-smi

Sun Oct 26 01:33:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
%%writefile main.py
# --- File main.py ---
# Phải chứa TẤT CẢ code: Tải API, Tải Data, VÀ Train

import os
from kaggle_secrets import UserSecretsClient # <--- Phải import l
from roboflow import Roboflow              # <--- Phải import lại
from rfdetr import RFDETRBase              # <--- Phải import lại

print("--- BẮT ĐẦU SCRIPT DDP TRÊN 2 GPU ---")

# 1. Lấy API Key (BÊN TRONG file main.py)
print("Đang tải API key...")
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("RB_Vehicle_1") 

# 2. Tải Dataset (BÊN TRONG file main.py)
print("Đang tải dataset 3.3k ảnh...")
rf = Roboflow(api_key=api_key) 

project = rf.workspace("myworkspace-jlyfn").project("vehicle-detection-byizq-i3hsv") 
version = project.version(1) 
dataset = version.download("coco")
DATASET_PATH = dataset.location
print(f"Dataset đã được tải về: {DATASET_PATH}")

# 3. Cấu hình Huấn luyện
OUTPUT_PATH = "/kaggle/working/RFDETR_Training_3.3k"
model = RFDETRBase()
print("Đã khởi tạo mô hình. Bắt đầu huấn luyện...")

# 4. Chạy Huấn luyện
model.train(
    dataset_dir = DATASET_PATH,
    output_dir = OUTPUT_PATH,
    epochs=300,
    batch_size = 4,      
    grad_accum_steps=4,
    lr=1e-4,
    tensorboard=True,
    early_stopping=True,
    early_stopping_patience=25, 
    early_stopping_min_delta=0.001 
)

print(f"--- HUẤN LUYỆN HOÀN TẤT ---")
print(f"Checkpoints được lưu tại: {OUTPUT_PATH}")
# --- Kết thúc file main.py ---

Writing main.py


In [5]:
!python -m torch.distributed.launch --nproc_per_node=2 main.py

/usr/local/lib/python3.11/dist-packages/torch/distributed/launch.py:208: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  main()
W1026 01:34:03.511000 82 torch/distributed/run.py:792] 
W1026 01:34:03.511000 82 torch/distributed/run.py:792] *****************************************
W1026 01:34:03.511000 82 torch/distributed/run.py:792] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1026 01:34:03.511000 82 torch/distributed/run.py:792] *****************************************
2025-10-26 01:34:23.49